# 1) Scrape jobs from company websites

## Setup: Import packages needed

In [261]:
# import packages needed for the code
import requests
import re
from lxml import html
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display

## a) N26 jobs

### Collect info from main jobs page

In [343]:
# navigate to & parse the company's main jobs page
page_N26 = requests.get('https://n26.com/en/careers').content
tree_N26 = html.fromstring(page_N26)

# get list of departments from main N26 careers page
dept_N26 = tree_N26.xpath('/html/body/div[1]/div/main/div[3]/div/nav/ul[1]/li/a/@href')
dept_N26 = pd.Series(dept_N26)
dept1_N26 = dept_N26.str.replace(pat='/en', repl='https://n26.com/en')

# reset variables
id_N26 = []
job_N26 = []
loc_N26 = []

# create function to pull postings from each department page
def f_main_N26(i):
    page1_N26 = requests.get(dept1_N26[i]).content
    tree1_N26 = html.fromstring(page1_N26)
    global id_N26
    global job_N26
    global loc_N26
    id_N26 += tree1_N26.xpath('//ul[@class="ah aj al an ap aq jj jy jz ka kb"]/li/a/@href')
    job_N26 += tree1_N26.xpath('//ul[@class="ah aj al an ap aq jj jy jz ka kb"]//li/a/div/text()')
    loc_N26 += tree1_N26.xpath('//dt[contains(text(), "Location")]/following-sibling::dd[1]/text()')
    return ([id_N26, job_N26, loc_N26])

# call the main dept page function and run for each page
for i in range (0, len(url_dept)):
    f_main_N26(i)
    
# clean names of locations
loc_N26 = pd.Series(loc_N26)
locClean_N26 = loc_N26.str.replace(pat='-+.*', repl='')
locClean_N26 = [re.sub('[^a-zA-Z0-9 ,./\-]+', '', x) for x in locClean_N26]
locClean_N26 = [x.rstrip() for x in locClean_N26]
locClean_N26 = pd.Series(locClean_N26)

# make a dataframe of all the job posting summary data pulled
df_N26 = pd.concat([pd.Series(id_N26), pd.Series(job_N26)
                , pd.Series(locClean_N26)]
                , axis=1)
                
df_N26.columns = ['id', 'title' 
              , 'location']

# drop any duplicated entries as the same posting could be under multiple departments
df_N26 = df_N26.drop_duplicates()

df_N26

,id,title,location
0,/en/careers/positions/2150159,Director of Banking Strategy and Excellence,Berlin
1,/en/careers/positions/1888137,IT Internal Auditor,Berlin
2,/en/careers/positions/2173762,IT Outsourcing Associate,Berlin
3,/en/careers/positions/1878930,"Outsourcing, Process and Relationship Manager ...",Berlin
4,/en/careers/positions/2061298,Sourcing Manager - Workplace services & real e...,Berlin
5,/en/careers/positions/1845515,Head of Global Learning Experience - Customer ...,Berlin
6,/en/careers/positions/1825819,Senior Data Engineer (DWH & Data Lake),Berlin
7,/en/careers/positions/1960867,Senior Data Engineer (Machine Learning Enginee...,Berlin
8,/en/careers/positions/1908639,Operations and Strategic Project Manager- Fran...,Berlin
9,/en/careers/positions/2072908,Finance Business Partner / Controller - Growth,Berlin


### Pull details from individual job postings

In [345]:
# function to convert job ids to urls
def f_url_N26(i):
    newid = df_N26.id[i]
    url = ('https://n26.com' + newid)
    return url

# call url function
url_N26 = []
for i in range (0, len(df_N26.id)):
    url_N26 += [f_url_N26(i)]

    
# reset variables
dept_N26 = []
desc_N26 = []

# create a function to capture dept & job description from individual pages #
def f_detail_N26(i):
    global dept_N26
    list_desc = ''
    page2_N26 = requests.get('https://n26.com' + df_N26.id[i]).content
    tree2_N26 = html.fromstring(page2_N26)
    dept_N26 += tree2_N26.xpath('//dt[contains(text(), "Department")]/following-sibling::dd/a/text()')
    list_desc = tree2_N26.xpath('//*[@id="main"]/div/div[3]/div/section/ul/li/text()')
    list_desc = [re.sub('[^a-zA-Z0-9 ,./\-]+', '', x) for x in list_desc]
    description = ''
    for j in range (0, len(list_desc)):
        description += (str(list_desc[j]) + str('/ '))
    return description

# call the function for each job posting page
for i in range (0, len(df_N26.id)):
    desc_N26 += [f_detail_N26(i)]

    
# create a column to register the date and time when info was pulled
now = datetime.now().strftime("%Y/%m/%d")
date = []
for i in range (0, len(df_N26.id)):
    date += [now]

now2 = datetime.now().strftime('%H:%M:%S')
time = []
for i in range (0, len(df_N26.id)):
    time += [now2]

# create a column to specify the company of the postings
comp_N26 = []
for i in range (0, len(df_N26.id)):
    comp_N26 += ['N26']

    
# pack info into a dataframe
df1_N26 = pd.concat([pd.Series(comp_N26)
                   , df_N26.title, df_N26.location
                   , pd.Series(dept_N26), pd.Series(desc_N26)
                   , pd.Series(url_N26)
                   , pd.Series(date), pd.Series(time)]
                  , axis=1)
df1_N26.columns = ['company'
                 , 'title', 'location'
                 , 'department', 'description'
                 , 'url'
                 , 'pull_date', 'pull_time']
df1_N26

,company,title,location,department,description,url,pull_date,pull_time
0,N26,Director of Banking Strategy and Excellence,Berlin,Banking,"Assume responsibility for the implementation, ...",https://n26.com/en/careers/positions/2150159,2020/05/04,19:07:40
1,N26,IT Internal Auditor,Berlin,Banking,"Untersttz uns dabei, eine unabhngige Interne K...",https://n26.com/en/careers/positions/1888137,2020/05/04,19:07:40
2,N26,IT Outsourcing Associate,Berlin,Banking,Define IT requirements for IT security. Initia...,https://n26.com/en/careers/positions/2173762,2020/05/04,19:07:40
3,N26,"Outsourcing, Process and Relationship Manager ...",Berlin,Banking,"You will be responsible for the outsourcing, m...",https://n26.com/en/careers/positions/1878930,2020/05/04,19:07:40
4,N26,Sourcing Manager - Workplace services & real e...,Berlin,Business Operations,Collaborate with stakeholders across Workplace...,https://n26.com/en/careers/positions/2061298,2020/05/04,19:07:40
5,N26,Head of Global Learning Experience - Customer ...,Berlin,Customer Operations,Develop an effective learning strategy at scal...,https://n26.com/en/careers/positions/1845515,2020/05/04,19:07:40
6,N26,Senior Data Engineer (DWH & Data Lake),Berlin,Data,Drive the development of a highly scalable Dat...,https://n26.com/en/careers/positions/1825819,2020/05/04,19:07:40
7,N26,Senior Data Engineer (Machine Learning Enginee...,Berlin,Data,Enable data scientists to develop robust machi...,https://n26.com/en/careers/positions/1960867,2020/05/04,19:07:40
8,N26,Operations and Strategic Project Manager- Fran...,Berlin,European Markets,Have an incredible opportunity to provide supp...,https://n26.com/en/careers/positions/1908639,2020/05/04,19:07:40
9,N26,Finance Business Partner / Controller - Growth,Berlin,Finance,"Be part of the Group FPA and Controlling team,...",https://n26.com/en/careers/positions/2072908,2020/05/04,19:07:40


-------------------

## b) Spotify jobs

### Get info from the main jobs page

In [348]:
# navigate to & parse the company's main jobs page
page_SP = requests.get('https://jobs.lever.co/spotify').content
tree_SP = html.fromstring(page_SP)

# capture info from the main job page
id_SP = tree_SP.xpath('//div/@data-qa-posting-id')
job_SP = tree_SP.xpath('//h5[@data-qa="posting-name"]/text()')
loc_SP = tree_SP.xpath('//span[@class="sort-by-location posting-category small-category-label"]/text()')
dept_SP = tree_SP.xpath('//span[@class="sort-by-team posting-category small-category-label"]/text()')

# clean names of locations to remove the state for US locations
loc_SP = pd.Series(loc_SP)
locClean_SP = loc_SP.str.replace(pat=',+.*', repl='')
locClean_SP = [re.sub('[^a-zA-Z0-9 ,./\-]+', '', x) for x in locClean_SP]
locClean_SP = [x.rstrip() for x in locClean_SP]
locClean_SP = pd.Series(locClean_SP)

# clean dept names to remove anything after the hyphen
dept_SP = pd.Series(dept_SP)
deptClean_SP = dept_SP.str.replace(pat=' –+.*', repl='')
deptClean_SP = pd.Series(deptClean_SP)

# create a dataframe with the high level information about jobs
df_SP = pd.concat([pd.Series(id_SP), pd.Series(job_SP)
                , locClean_SP, deptClean_SP]
                , axis=1)
                
df_SP.columns = ['id', 'title' 
              , 'location', 'department']

# check for duplicate entries
df_SP['id'].duplicated().sum()

# drop any duplicate entries if any
df_SP = df_SP.drop_duplicates()

df_SP

,id,title,location,department
0,22ca9c88-1a3a-4caf-a216-a40386f3e129,Backend Engineer - Podcaster Mission,New York,Anchor
1,5ac94b16-fc1d-4c0c-b0e0-6ae066b2a512,Director of Engineering - Podcaster Mission,New York,Anchor
2,dedbf9a9-d95c-4741-8e2b-b4225b6aa2be,Group Product Manager - Podcaster Mission,New York,Anchor
3,7aa2304e-aee6-4811-804a-1df4049f6af6,International Creator Growth Manager - Podcast...,New York,Anchor
4,1f535e9a-56a6-4ee0-814b-152db16cfbdf,Senior Web Engineer - Podcaster Mission,New York,Anchor
...,...,...,...,...
275,d0490a51-f4d4-4e27-bd93-4e687faed072,"Senior Sales Executive, Education",New York,Soundtrap
276,01833a69-f385-4b5e-9a79-dbfeb6eb7d7c,Software Developer - Growth Initiatives for th...,Stockholm,Soundtrap
277,83e9dcd7-f3b4-4aaa-9592-fe8fe198e8fb,"Software Developer, Creator - for the Soundtra...",Stockholm,Soundtrap
278,5de1999e-91ad-4897-b2f4-456845f65712,"Software Developer, Education - for the Soundt...",Stockholm,Soundtrap


### Pull details from individual job postings

In [360]:
# create function to pull urls for job postings
def f_url_SP(i):
    newid = df_SP.id[i]
    url = ('https://jobs.lever.co/spotify/' + newid)
    return url

# call the url function
url_SP = []
for i in range (0, len(df_SP.id)):
    url_SP += [f_url_SP(i)]
    
# create a function to pull information from individual job pages
def f_detail_SP(i):
    page1_SP = requests.get('https://jobs.lever.co/spotify/' + df_SP.id[i]).content
    tree1_SP = html.fromstring(page1_SP)
    list_desc = tree1_SP.xpath('//ul[@class="posting-requirements plain-list"]/ul/li/text()')
    list_desc = [re.sub('[^a-zA-Z0-9 ,./\-]+', '', x) for x in list_desc]
    description = ''
    for j in range (0, len(list_desc)):
        description += (str(list_desc[j]) + str('/ '))
    return description

# call the function for detailed page
desc_SP = []
for i in range (0, len(df_SP.id)):
    desc_SP += [f_detail_SP(i)]
    
# create a column for the date & time postings were pulled
now = datetime.now().strftime("%Y/%m/%d")
date = []
for i in range (0, len(df_SP.id)):
    date += [now]

now2 = datetime.now().strftime('%H:%M:%S')
time = []
for i in range (0, len(df_SP.id)):
    time += [now2]

# create a column for the company name
comp_SP = []
for i in range (0, len(df_SP.id)):
    comp_SP += ['Spotify']
    
# pack information into new dataframe
df1_SP = pd.concat([pd.Series(comp_SP)
                   , df_SP.title, df_SP.location
                   , df_SP.department, pd.Series(desc_SP)
                   , pd.Series(url_SP)
                   , pd.Series(date), pd.Series(time)]
                  , axis=1)
df1_SP.columns = ['company'
                 , 'title', 'location'
                 , 'department', 'description'
                 , 'url'
                 , 'pull_date', 'pull_time']
df1_SP

,company,title,location,department,description,url,pull_date,pull_time
0,Spotify,Backend Engineer - Podcaster Mission,New York,Anchor,Build out the . server that powers Anchors mo...,https://jobs.lever.co/spotify/22ca9c88-1a3a-4c...,2020/05/04,19:30:12
1,Spotify,Director of Engineering - Podcaster Mission,New York,Anchor,Be the senior technical leader for the Podcast...,https://jobs.lever.co/spotify/5ac94b16-fc1d-4c...,2020/05/04,19:30:12
2,Spotify,Group Product Manager - Podcaster Mission,New York,Anchor,We are looking for a passionate and experience...,https://jobs.lever.co/spotify/dedbf9a9-d95c-47...,2020/05/04,19:30:12
3,Spotify,International Creator Growth Manager - Podcast...,New York,Anchor,Own and execute Anchors international growth s...,https://jobs.lever.co/spotify/7aa2304e-aee6-48...,2020/05/04,19:30:12
4,Spotify,Senior Web Engineer - Podcaster Mission,New York,Anchor,Construct product features and templates from ...,https://jobs.lever.co/spotify/1f535e9a-56a6-4e...,2020/05/04,19:30:12
...,...,...,...,...,...,...,...,...
275,Spotify,"Senior Sales Executive, Education",New York,Soundtrap,Manage and close sales opportunities on the US...,https://jobs.lever.co/spotify/d0490a51-f4d4-4e...,2020/05/04,19:30:12
276,Spotify,Software Developer - Growth Initiatives for th...,Stockholm,Soundtrap,Were looking for an experienced web developer ...,https://jobs.lever.co/spotify/01833a69-f385-4b...,2020/05/04,19:30:12
277,Spotify,"Software Developer, Creator - for the Soundtra...",Stockholm,Soundtrap,Work with modern Java in the backend to archit...,https://jobs.lever.co/spotify/83e9dcd7-f3b4-4a...,2020/05/04,19:30:12
278,Spotify,"Software Developer, Education - for the Soundt...",Stockholm,Soundtrap,"Youll be joining Soundtraps education team, fo...",https://jobs.lever.co/spotify/5de1999e-91ad-48...,2020/05/04,19:30:12


-------------------

# 2) Save output as files

## a) Concat jobs in one dataframe

In [307]:
# concatenate job postings from all companies
df = pd.concat([df1_SP, df1_N26]
               , axis=0)

# reset the index
df = df.reset_index(drop=True)
df

,company,title,location,department,description,url,pull_date,pull_time
0,Spotify,Backend Engineer - Podcaster Mission,New York,Anchor,Build out the . server that powers Anchors mo...,https://jobs.lever.co/spotify/22ca9c88-1a3a-4c...,2020/05/04,18:24:18
1,Spotify,Director of Engineering - Podcaster Mission,New York,Anchor,Be the senior technical leader for the Podcast...,https://jobs.lever.co/spotify/5ac94b16-fc1d-4c...,2020/05/04,18:24:18
2,Spotify,Group Product Manager - Podcaster Mission,New York,Anchor,We are looking for a passionate and experience...,https://jobs.lever.co/spotify/dedbf9a9-d95c-47...,2020/05/04,18:24:18
3,Spotify,International Creator Growth Manager - Podcast...,New York,Anchor,Own and execute Anchors international growth s...,https://jobs.lever.co/spotify/7aa2304e-aee6-48...,2020/05/04,18:24:18
4,Spotify,Senior Web Engineer - Podcaster Mission,New York,Anchor,Construct product features and templates from ...,https://jobs.lever.co/spotify/1f535e9a-56a6-4e...,2020/05/04,18:24:18
...,...,...,...,...,...,...,...,...
303,N26,Principal Engineer - Assistance and Identity,Berlin,Technology,Build a platform for back-office operations th...,https://n26.com/en/careers/positions/2111328,2020/05/04,18:16:11
304,N26,Senior Product Security Engineer,Barcelona,Technology,Use penetration testing skills and methodology...,https://n26.com/en/careers/positions/1817199,2020/05/04,18:16:11
305,N26,Senior Security Engineer - Infrastructure,Berlin,Technology,"Use software engineering skills to design, bui...",https://n26.com/en/careers/positions/1768013,2020/05/04,18:16:11
306,N26,Systems Engineer - AML Solution,Berlin,Technology,Integrate and maintain the new provided infras...,https://n26.com/en/careers/positions/2104215,2020/05/04,18:16:11


## b) Setup: directories and file names

In [91]:
# set directories where files should be saved & retrieved from
## 'old' file contained in the main folder - need double slashes \\ to make it read correctly
dirMain = 'C:\\Users\\Madisyn\\Desktop\\ISP\\Data\\' 
## save the current file in the Archive folder for future reference
dirArchive = 'C:\\Users\\Madisyn\\Desktop\\ISP\\Data\\Archive\\'

# set up variable for today's date to save in filename
today = datetime.now().strftime("%Y%m%d")
fname_wdate = dirArchive + 'ALL_' + today
fname_wdate

# set filenames for old and new files in the main data folder
fname_oldj = dirMain + 'ALL_old.json'
fname_newj = dirMain + 'ALL_new.json'

## c) Archive current df with today's date

In [94]:
# button to save current df as file in Archive
## create button and label widgets
btn_saveArch = widgets.Button(description='Save to archive')
txt_saveArch1 = widgets.Label(value ='')

## define a function on button click
def btn_saveArch_click(b):
    df.to_json (fname_wdate + '.json')
    df.to_csv(fname_wdate + '.csv', sep=';', index=False)
    txt_saveArch1 = widgets.Label(value ='File saved!')
    display(txt_saveArch1)
    
btn_saveArch.on_click(btn_saveArch_click)

## display box widget with button and text
box1 = widgets.HBox([widgets.Label(value ='Save current df as json & csv:')
                        , btn_saveArch]
                        , layout={'width' : 'max-content'} )
display(box1)

Label(value='File saved!')

## d) Convert the new file to old

In [102]:
# click button to make new.json --> old.json
btn_newToOld = widgets.Button(description='Execute')
txt_newToOld_conf = widgets.Label(value ='')

def btn_newToOld_click(b):
    df_temp = pd.read_json(fname_newj)
    df_temp.to_json(fname_oldj)
    txt_newToOld_conf = widgets.Label(value ='Converted!')
    display(txt_newToOld_conf)
    
btn_newToOld.on_click(btn_newToOld_click)

box2 = widgets.HBox([widgets.Label(value ='Convert new.json to old.json:')
                        , btn_newToOld]
                        , layout={'width': 'max-content'})
display(box2)

Label(value='Converted!')

### c) Choose either new.json or old.json to import as df_old

In [104]:
# reset df_old to 0 
df_old = []

# selection and button to import old dataframe
## set variables and create widgets to choose which file to import
Old = 'Old.json'
New = 'New.json'

output = widgets.Output()
wid_fileChoose = widgets.Dropdown(options=[Old, New], description='Choose file:')
btn_impOld = widgets.Button(description='Import as df_old')
txt_impOld = widgets.Label(value ='')

## create function to choose which file to import
## define button click action
def btn_impOld_click(b):
    output.clear_output()
    fileOldNew = wid_fileChoose.value
    txt_impOld = widgets.Label(value ='File imported as df_old')
    global df_old
    if (fileOldNew == Old):
        df_old = pd.read_json(fname_oldj)
    else:
        df_old = pd.read_json(fname_newj)
    with output:
        display(txt_impOld)
        display(df_old)
    return df_old
    
btn_impOld.on_click(btn_impOld_click)

## display widgets and output
box3 = widgets.HBox([wid_fileChoose
                         , btn_impOld])
display(box3)
display(output)

Output()

### d) Save current df as the new file 

In [105]:
# click button to save current df as new file
btn_saveNew = widgets.Button(description='Execute')
txt_saveNew_conf = widgets.Label(value ='')

def btn_saveNew_click(b):
    df.to_json (fname_newj)
    txt_saveNew_conf = widgets.Label(value ='Files saved as new.json!')
    display(txt_saveNew_conf)
    
btn_saveNew.on_click(btn_saveNew_click)

out_box3 = widgets.HBox([widgets.Label(value ='Save df as new.json:')
                        , btn_saveNew]
                        , layout={'width': 'max-content'})
display(out_box3)

Label(value='Files saved as new.json!')

-------------------

# 3) Filtering & selecting information

## Setup: determine which postings are old vs. new

In [339]:
# Make series of urls from "old" dataframe
df_old_url = df_old['url']

# join the old & current dataframes to see which entries are new
df_all = pd.merge(df, df_old_url, 
                  on='url',
                  how='left', indicator=True)

# rename the _merge column to status
df_all = df_all.rename(columns={'_merge' : 'status'})

# replace those marked "left-join" with "new", and "both" with "all" to indicate if this is new job
df_all['status'] = df_all['status'].str.replace('left_only','New')
df_all['status'] = df_all['status'].str.replace('both','Old')
df_all

,company,title,location,department,description,url,pull_date,pull_time,status
0,Spotify,Backend Engineer - Podcaster Mission,New York,Anchor,Build out the . server that powers Anchors mo...,https://jobs.lever.co/spotify/22ca9c88-1a3a-4c...,2020/05/04,18:24:18,Old
1,Spotify,Director of Engineering - Podcaster Mission,New York,Anchor,Be the senior technical leader for the Podcast...,https://jobs.lever.co/spotify/5ac94b16-fc1d-4c...,2020/05/04,18:24:18,Old
2,Spotify,Group Product Manager - Podcaster Mission,New York,Anchor,We are looking for a passionate and experience...,https://jobs.lever.co/spotify/dedbf9a9-d95c-47...,2020/05/04,18:24:18,Old
3,Spotify,International Creator Growth Manager - Podcast...,New York,Anchor,Own and execute Anchors international growth s...,https://jobs.lever.co/spotify/7aa2304e-aee6-48...,2020/05/04,18:24:18,Old
4,Spotify,Senior Web Engineer - Podcaster Mission,New York,Anchor,Construct product features and templates from ...,https://jobs.lever.co/spotify/1f535e9a-56a6-4e...,2020/05/04,18:24:18,Old
...,...,...,...,...,...,...,...,...,...
303,N26,Principal Engineer - Assistance and Identity,Berlin,Technology,Build a platform for back-office operations th...,https://n26.com/en/careers/positions/2111328,2020/05/04,18:16:11,New
304,N26,Senior Product Security Engineer,Barcelona,Technology,Use penetration testing skills and methodology...,https://n26.com/en/careers/positions/1817199,2020/05/04,18:16:11,New
305,N26,Senior Security Engineer - Infrastructure,Berlin,Technology,"Use software engineering skills to design, bui...",https://n26.com/en/careers/positions/1768013,2020/05/04,18:16:11,New
306,N26,Systems Engineer - AML Solution,Berlin,Technology,Integrate and maintain the new provided infras...,https://n26.com/en/careers/positions/2104215,2020/05/04,18:16:11,New


## a) Filter jobs by company, type, location, keyword

In [335]:
# set variables for all and new only
ALL1 = 'ALL'
New2 = 'New only'

# function to make list of unique entries for drop downs
def list_unique_sorted_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL1)
    return unique

# create widgets for jobs and location dropdowns
output = widgets.Output()

wid_stat = widgets.Dropdown(options = [ALL1, New2], description='Posting status:')
wid_comp = widgets.Dropdown(options = list_unique_sorted_ALL(df_all.company), description='Company:' )
wid_loc = widgets.Dropdown(options = list_unique_sorted_ALL(df_all.location), description='Location:')
wid_pos = widgets.Text(description='Job title:', placeholder='Enter a keyword to search')
wid_desc = widgets.Text(description='Job description:', placeholder='Enter a keyword to search')
btn_filter = widgets.Button(description='Run')

# create function to filger based on the resulting information
def filtering1(new, comp, loc, pos, desc):
    output.clear_output()
    pos = pos.lower()
    desc = desc.lower()
    global filter1
    filter1 = []
    if (new == ALL1) & (loc == ALL1) & (comp == ALL1):
        filter1 = df_all.loc[(df_all.title.str.lower().str.contains(pos)) 
                         & (df_all.description.str.lower().str.contains(desc))
                        ,['company', 'title', 'location', 'description', 'url']]
    elif (loc == ALL1) & (comp == ALL1):
        filter1 = df_all.loc[(df_all.status == 'New') 
                         & (df_all.title.str.lower().str.contains(pos)) 
                         & (df_all.description.str.lower().str.contains(desc))
                        ,['company', 'title', 'location', 'description', 'url']]
    elif (new == ALL1) & (comp == ALL1): 
        filter1 = df_all.loc[(df_all.location == loc) 
                         & (df_all.title.str.lower().str.contains(pos)) 
                         & (df_all.description.str.lower().str.contains(desc))
                        ,['company', 'title', 'location', 'description', 'url']]
    elif (new == ALL1) & (loc == ALL1): 
        filter1 = df_all.loc[(df_all.company == comp) 
                         & (df_all.title.str.lower().str.contains(pos)) 
                         & (df_all.description.str.lower().str.contains(desc))
                        ,['company', 'title', 'location', 'description', 'url']]
    elif (new == ALL1): 
        filter1 = df_all.loc[(df_all.company == comp) 
                         & (df_all.location == loc) 
                         & (df_all.title.str.lower().str.contains(pos)) 
                         & (df_all.description.str.lower().str.contains(desc))
                        ,['company', 'title', 'location', 'description', 'url']]
    elif (comp == ALL1): 
        filter1 = df_all.loc[(df_all.status == 'New') 
                         & (df_all.location == loc) 
                         & (df_all.title.str.lower().str.contains(pos)) 
                         & (df_all.description.str.lower().str.contains(desc))
                         ,['company', 'title', 'location', 'description', 'url']] 
    elif (loc == ALL1): 
        filter1 = df_all[(df_all.status == 'New') 
                         & (df_all.company == comp) 
                         & (df_all.title.str.lower().str.contains(pos)) 
                         & (df_all.description.str.lower().str.contains(desc))]
    else:
        filter1 = df_all.loc[(df_all.status == 'New') 
                         & (df_all.company == comp) 
                         & (df_all.location == loc) 
                         & (df_all.title.str.lower().str.contains(pos)) 
                         & (df_all.description.str.lower().str.contains(desc))
                        ,['company', 'title', 'location', 'description', 'url']]
    with output:
        print(str(len(filter1.title)) + ' jobs found')
        display(filter1[['company', 'title', 'location', 'description']])
    return filter1


def btn_filter_click(b):
    filtering1(wid_stat.value, wid_comp.value
               , wid_loc.value, wid_pos.value
               , wid_desc.value) 
    
    
btn_filter.on_click(btn_filter_click)

box4 = widgets.VBox([wid_stat, wid_comp
                        , wid_loc, wid_pos
                        , wid_desc
                        , btn_filter]
                        , layout={'width' : 'max-content'})
display(box4)
display(output)

Output()

## b) Save the results as csv and json file

In [387]:
# Set df_out to the current output of the filters
df_out = filter1.reset_index(drop=True)

# set up variable for today's date to save in filename
dirScript = 'C:\\Users\\Madisyn\\Desktop\\ISP\\' 
today = datetime.now().strftime("%Y%m%d")
fname_out = dirMain + 'Filtered_' + today

# button to save current info as output file
## create button and label widgets
btn_saveOut = widgets.Button(description='Save as Excel')
txt_saveOut = widgets.Label(value ='')

## define a function on button click
def btn_saveOut_click(b):
#     df_out.to_json (fname_out + '.json')
#     df_out.to_csv(fname_out + '.csv', sep=';', index=False)
    df_out.to_excel(fname_out + '.xlsx', index=False)
    txt_saveOut = widgets.Label(value ='Saved as Output file!')
    display(txt_saveOut)
    
btn_saveOut.on_click(btn_saveOut_click)

## display box widget with button and text
box5 = widgets.HBox([widgets.Label(value ='Save current info as file:')
                        , btn_saveOut]
                        , layout={'width' : 'max-content'} )
display(box5)

Label(value='Saved as Output file!')

# 4) Save new job postings and send as email

# Save new postings as Excel file

In [385]:
# make dataframe will only the new postings
df_newOnly = df_all.loc[(df_all.status == 'New') 
                        ,['company', 'title', 'location', 'description', 'url']].reset_index(drop=True)

# set up variable for today's date to save in filename
dirScript = 'C:\\Users\\Madisyn\\Desktop\\ISP\\'
today = datetime.now().strftime("%Y%m%d")
fname_newOnly = dirScript + 'NewPostings'

# button to save current info as output file
## create button and label widgets
btn_newOnly = widgets.Button(description='Save as Excel')
txt_newOnly = widgets.Label(value ='')

## define a function on button click
def btn_newOnly_click(b):
    df_newOnly.to_excel(fname_newOnly + '.xlsx', index=False)
    txt_newOnly = widgets.Label(value ='Saved as New Only file!')
    display(txt_newOnly)
    
btn_newOnly.on_click(btn_newOnly_click)

## display box widget with button and text
box6 = widgets.HBox([widgets.Label(value ='Save new postings to file:')
                        , btn_newOnly]
                        , layout={'width' : 'max-content'} )
display(box6)

Label(value='Saved as New Only file!')

# Send email with new postings

In [390]:
import email, smtplib, ssl

from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

subject = ("New job postings - " + str(today))
# body = "This is an email with attachment sent from Python"
sender_email = "n00batstartups@gmail.com"
receiver_email = "n00batstartups@gmail.com"
password = "v*$80Hi%eA"

# Create a multipart message and set headers
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = subject
# message["Bcc"] = receiver_email  # Recommended for mass emails

# Add body to email
# message.attach(MIMEText(body, "plain"))
html = """\
<html>
  <body>
    <p>Hi!<br>
           Here are the new job postings:<br>
           {0}
    </p>
  </body>
</html>
""".format(df_new.to_html())

partHTML = MIMEText(html, 'html')
message.attach(partHTML)

filename = str(fname_newOnly) + '.xlsx'  # In same directory as script

# Open PDF file in binary mode
with open(filename, "rb") as attachment:
    # Add file as application/octet-stream
    # Email client can usually download this automatically as attachment
    part = MIMEBase("application", "octet-stream")
    part.set_payload(attachment.read())

# Encode file in ASCII characters to send by email    
encoders.encode_base64(part)

# Add header as key/value pair to attachment part
part.add_header(
    "Content-Disposition",
    f"attachment; filename= {filename}",
)

# Add attachment to message and convert message to string
message.attach(part)
text = message.as_string()

# Log in to server using secure context and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, text)